In [1]:

import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests
from tensorflow.python.tools import inspect_checkpoint as chkp
import os
import scipy.misc
import numpy as np

In [2]:
# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))


TensorFlow Version: 1.13.1
Default GPU Device: /device:GPU:0


In [4]:
def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
           
    tf.saved_model.loader.load(sess,[vgg_tag],vgg_path)  #load vgg as default graph
    detection_graph = tf.get_default_graph()             # get the default graph
    
    #Extract Tensors
    image_input = detection_graph.get_tensor_by_name(vgg_input_tensor_name)    
    keep_prob = detection_graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    layer3_out = detection_graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    layer4_out = detection_graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    layer7_out = detection_graph.get_tensor_by_name(vgg_layer7_out_tensor_name)
    
    return (image_input, keep_prob, layer3_out, layer4_out, layer7_out)

tests.test_load_vgg(load_vgg, tf)


Tests Passed


In [5]:

def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer3_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer7_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    
    conv7_1x1 = tf.layers.conv2d(vgg_layer7_out,
                                 num_classes,1, 
                                 padding='same', 
                                 kernel_initializer = tf.random_normal_initializer(stddev=0.01),
                                 kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3), 
                                 name = 'conv7_1x1')
    
    upsample1 = tf.layers.conv2d_transpose(conv7_1x1, 
                                           num_classes,
                                           4,
                                           strides=(2, 2),
                                           padding='same', 
                                           kernel_initializer = tf.random_normal_initializer(stddev=0.01),
                                           kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3), name = 'upsample1')
    
    conv4_1x1 = tf.layers.conv2d(vgg_layer4_out, 
                                 num_classes,
                                 1, 
                                 padding='same',
                                 kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3),
                                 kernel_initializer = tf.random_normal_initializer(stddev=0.01),
                                 name = 'conv4_1x1') 
    
    skip1 = tf.add(upsample1, conv4_1x1, name = 'skip1')
    
    
    
    
    
    upsample2 = tf.layers.conv2d_transpose(skip1,
                                           num_classes,
                                           4,
                                           strides=(2, 2),
                                           padding='same', 
                                           kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3), 
                                           name = 'upsample2') 
    
    conv3_1x1 = tf.layers.conv2d(vgg_layer3_out, 
                                 num_classes,1, 
                                 padding='same',
                                 kernel_initializer = tf.random_normal_initializer(stddev=0.01),
                                 kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3), name = 'conv3_1x1')  
    
    skip2 = tf.add(upsample2, conv3_1x1, name = 'skip2')
    
    upsample3 = tf.layers.conv2d_transpose(skip2, 
                                           num_classes,
                                           16,
                                           strides=(8, 8), 
                                           padding='same', 
                                           kernel_initializer = tf.random_normal_initializer(stddev=0.01),
                                           kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3), 
                                           name = 'upsample3') 
   
    
    return upsample3

tests.test_layers(layers)



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.conv2d_transpose instead.
Tests Passed


In [6]:
def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    # TODO: Implement function

    logits = tf.reshape(nn_last_layer, (-1, num_classes))
    correct_label = tf.reshape(correct_label, (-1,num_classes))
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= logits, labels= correct_label))
    optimizer = tf.train.AdamOptimizer(learning_rate= learning_rate)
    train_op = optimizer.minimize(cross_entropy_loss)
    
    return  (logits, train_op, cross_entropy_loss)

tests.test_optimize(optimize)


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Tests Passed


In [7]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    
    sess.run(tf.global_variables_initializer())

    print('Starting training... for {} epochs'.format(epochs))
    print()
    for epoch in range(epochs):
        print('Epoch : {}'.format(epoch + 1))
        loss_log = []
        for image, label in get_batches_fn(batch_size):
            _, loss = sess.run([train_op, cross_entropy_loss],
                                feed_dict={
                                    input_image: image,
                                    correct_label: label,
                                    keep_prob: 0.5,
                                    learning_rate: 0.00001
                                })
            loss_log.append('{:3f}'.format(loss))
        print(loss_log)
        print("----------------------------------------------")
    print('Training finished')
    
tests.test_train_nn(train_nn)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./data\vgg\variables\variables
INFO:tensorflow:Restoring parameters from .\fcnmodel
INFO:tensorflow:Restoring parameters from .\fcnmodel
Instructions for updating:
Please use tf.global_variables instead.


In [10]:
def run():
    num_classes = 2
    image_shape = (160, 576)  # KITTI dataset uses 160x576 images
    data_dir = './data'
    runs_dir = './runs'
    tests.test_for_kitti_dataset(data_dir)
    
    helper.maybe_download_pretrained_vgg(data_dir)

    with tf.Session() as sess:
     
        vgg_path = os.path.join(data_dir, 'vgg')    
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)
        correct_label = tf.placeholder(tf.int32, [None, None, None, num_classes], name='correct_label')
        learning_rate = tf.placeholder(tf.float32, name='learning_rate')
        input_image, keep_prob, layer3_out, layer4_out, layer7_out = load_vgg(sess, vgg_path)
        layer_output = layers(layer3_out, layer4_out, layer7_out, num_classes)
        logits, train_op, cross_entropy_loss = optimize(layer_output, correct_label, learning_rate, num_classes)
        epochs = 32
        batch_size = 4
        saver = tf.train.Saver()
        train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
                 correct_label, keep_prob, learning_rate)
        
        saver.save(sess, 'fcnmodel')
        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)

In [12]:
run()

In [2]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [4]:
def pipeline(img):
    
    img_orig_size = (img.shape[0], img.shape[1])
    num_classes = 2
    image_shape = (160, 576)  # KITTI dataset uses 160x576 images
    
    detection_graph = tf.get_default_graph()
    image_input = detection_graph.get_tensor_by_name('image_input:0')    
    keep_prob = detection_graph.get_tensor_by_name('keep_prob:0')
    upsample3 = detection_graph.get_tensor_by_name('upsample3/conv2d_transpose:0')
    logits = tf.reshape(upsample3, (-1, num_classes))
    resized_image = scipy.misc.imresize(img, image_shape)
    
  
    im_softmax = sess.run(
            [tf.nn.softmax(logits)],
            {keep_prob: 1.0, image_input: [resized_image]})
    
    
    im_softmax = im_softmax[0][:, 1].reshape(image_shape[0], image_shape[1])
    segmentation = (im_softmax > 0.5).reshape(image_shape[0], image_shape[1], 1)
    mask = np.dot(segmentation, np.array([[0, 255, 0, 127]]))
    mask = scipy.misc.toimage(mask, mode="RGBA")
    street_im = scipy.misc.toimage(resized_image)
    street_im.paste(mask, box=None, mask=mask)
    street_img_resized = scipy.misc.imresize(street_im, img_orig_size)
    
    return np.array(street_img_resized)

In [5]:

sess = tf.Session() 
saver = tf.train.import_meta_graph('fcnmodel.meta')
saver.restore(sess, tf.train.latest_checkpoint('.\\'))

#Test Sample Image
    # graph = tf.get_default_graph()
    # list_of_tuples = [op.values() for op in graph.get_operations()]
    # list_of_tuples
    # image_file = "F:\\Data Science\\Semantic Segmentation\\CarND-Semantic-Segmentation-master\\data\\data_road\\testing\\image_2\\um_000000.png"
    # image_shape = (160, 576) 
    # image = scipy.misc.imresize(scipy.misc.imread(image_file), image_shape)
    # image.shape
    # pipeline(image)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from .\fcnmodel


In [6]:
clip = VideoFileClip('driving.mp4')
new_clip = clip.fl_image(pipeline)
new_clip.write_videofile('result.mp4')

Moviepy - Building video result.mp4.
MoviePy - Writing audio in %s


MoviePy - Done.
Moviepy - Writing video result.mp4



Moviepy - Done !
Moviepy - video ready result.mp4


In [3]:

HTML("""
<video width="960" height="600" controls>
  <source src="{0}" type="video/mp4">
</video>
""".format('result.mp4'))